In [ ]:
import cv2
import numpy as np
import roypy
import matplotlib.pyplot as plt
import argparse
import time
import queue
import math
import random
import matplotlib
import threading
from PIL import Image, ImageDraw
from sample_camera_info import print_camera_info
from roypy_sample_utils import CameraOpener, add_camera_opener_options

import get_clear_hand_mask
import find_fingertips
import tracking
import touched_detection
import socket_sender

try:
    import roypycy
except ImportError:
    print("Pico Flexx backend requirements (roypycy) not installed properly")
    raise

# 調整np.array輸出格式
np.set_printoptions(suppress=True)

# Image Streamer

In [ ]:
class MyListener(roypy.IDepthDataListener):
    data_lock = None
    store_data_threads = []
    store_thread_lock = threading.Lock()
    def __init__(self, z_queue, gray_queue, points3D_queue, ConfidenceIndex_queue, Confidence_queue, undistortImage=False):
        super(MyListener, self).__init__()
        self.z_queue = z_queue
        self.gray_queue = gray_queue
        self.points3D_queue = points3D_queue
        self.ConfidenceIndex_queue = ConfidenceIndex_queue
        self.Confidence_queue = Confidence_queue
        self.undistortImage = undistortImage
        self.cameraMatrix = None
        self.distortionCoefficients = None
        self.First = True
    
    def set_lock(self, lock):
        self.data_lock = lock

    def get_store_threads(self):
        return self.store_data_threads
        
    def onNewData(self, data):
        t = threading.Thread(target=self.store_data, args=(data,))
        t.setDaemon(False)
        
        try:
            # unregister the store thread
            self.store_thread_lock.acquire()
            # update store thread list
            for thread in self.store_data_threads:
                if not thread.is_alive():
                    self.store_data_threads.remove(thread)
            # store infroamtion
            self.store_data_threads.append(t)
            t.start()
        except Exception as e:
            print(e)
        self.store_thread_lock.release()

    def paint(self, data, name, isGray = False, cv2_show = False):
        """
        Called in the main thread, with data containing one of the items that was added to the queue in onNewData
        :param data:
        :return:
        """
        if cv2_show == True: 
            cv2.namedWindow(name, cv2.WINDOW_NORMAL)
            cv2.resizeWindow(name, 224*2,171*2)
            cv2.imshow(name, data)
        else:
            # create a figure and show the raw data
            plt.figure(1)
            if isGray == False:
                plt.imshow(data),plt.title(name),plt.axis('off')
            else: 
                plt.imshow(data,plt.cm.gray),plt.title(name),plt.axis('off')

            plt.show(block=False)
            plt.draw()

            # this pause is needed to ensure the drawing for some backends
            plt.pause(0.001)
    def store_data(self, data):
        s_time = time.time()
        zvalues = []
        xvalues = []
        yvalues = []
        grayvalues = []
        points3D = []
        ConfidenceIndexvalues = [] # whether the pixel measured a valid 3D value

        self.store_thread_lock.acquire()
        try:
            depth_data = roypycy.get_backend_data(data)
            xvalues = depth_data.x
            yvalues = depth_data.y
            zvalues = depth_data.z
            grayvalues = depth_data.grayValue
            confidences = depth_data.depthConfidence
            for i in range(len(confidences)):    
                if ((self.First == True) and (confidences[i] > 0)):
                    ConfidenceIndexvalues.append(i)
        except Exception as e:
            print(e)
        self.store_thread_lock.release()
        
        try:
            xarray = np.asarray(xvalues)
            yarray = np.asarray(yvalues)
            zarray = np.asarray(zvalues)
            x = xarray.reshape (-1, data.width)
            y = yarray.reshape (-1, data.width)
            z = zarray.reshape (-1, data.width)
            q = grayvalues.reshape (-1, data.width)
            points3D = np.dstack((x,y,z))
            if self.First == True:
                ConfidenceIndex = np.asarray(ConfidenceIndexvalues)
        except Exception as e:
            print(e)
        
        # save data
        try:
            self.data_lock.acquire()
            self.z_queue.put(z)
            self.points3D_queue.put(points3D)
            self.gray_queue.put(q)
            if self.First == True:
                self.ConfidenceIndex_queue.put(ConfidenceIndex)
        except Exception as e:
            print(e)
        self.data_lock.release()
        print('queue: %.4f'%(time.time() - s_time))

In [ ]:
# class MyListener(roypy.IDepthDataListener):

#     def __init__(self, z_queue, gray_queue, points3D_queue, ConfidenceIndex_queue, Confidence_queue, undistortImage=False):
#         super(MyListener, self).__init__()
#         self.z_queue = z_queue
#         self.gray_queue = gray_queue
#         self.points3D_queue = points3D_queue
#         self.ConfidenceIndex_queue = ConfidenceIndex_queue
#         self.Confidence_queue = Confidence_queue
#         self.undistortImage = undistortImage
#         self.cameraMatrix = None
#         self.distortionCoefficients = None
#         self.First = True

#     def onNewData(self, data):
#         s_time = time.time()
#         zvalues = []
#         xvalues = []
#         yvalues = []
#         grayvalues = []
#         points3D = []
#         ConfidenceIndexvalues = [] # whether the pixel measured a valid 3D value
#         Confidencevalue = []
        
#         points = data.points()
        
#         for i in range(data.getNumPoints()):
#             zvalues.append(points[i].z)
#             xvalues.append(points[i].x)
#             yvalues.append(points[i].y)
#             grayvalues.append(points[i].grayValue)
            
# #             Confidencevalue.append(data.getDepthConfidence(i))
#             if self.First == True:
#                 if points[i].depthConfidence > 0:
#                     ConfidenceIndexvalues.append(i)
            
#         zarray = np.asarray(zvalues)
#         z = zarray.reshape (-1, data.width)        
#         self.z_queue.put(z)
        
#         xarray = np.asarray(xvalues)
#         x = xarray.reshape (-1, data.width)
        
#         yarray = np.asarray(yvalues)
#         y = yarray.reshape (-1, data.width)
        
#         points3D = np.dstack((x,y,z))
#         self.points3D_queue.put(points3D)
        
        
#         grayarray = np.asarray(grayvalues)
#         q = grayarray.reshape (-1, data.width)        
#         self.gray_queue.put(q)
        
# #         Confidencearray = np.asarray(Confidencevalue)
# #         Confidence = Confidencearray.reshape (-1, data.width)
# #         self.Confidence_queue.put(Confidence)
#         #ConfidenceIndex_queue
#         if self.First == True:
#             ConfidenceIndex = np.asarray(ConfidenceIndexvalues)
#             self.ConfidenceIndex_queue.put(ConfidenceIndex)
            
#         print('queue: %.4f'%(time.time() - s_time))
# #         self.First = False

#     def paint(self, data, name, isGray = False, cv2_show = False):
#         """
#         Called in the main thread, with data containing one of the items that was added to the queue in onNewData
#         :param data:
#         :return:
#         """
#         if cv2_show == True: 
#             cv2.namedWindow(name, cv2.WINDOW_NORMAL)
#             cv2.resizeWindow(name, 224*2,171*2)
#             cv2.imshow(name, data)
#         else:
#             # create a figure and show the raw data
#             plt.figure(1)
#             if isGray == False:
#                 plt.imshow(data),plt.title(name),plt.axis('off')
#             else: 
#                 plt.imshow(data,plt.cm.gray),plt.title(name),plt.axis('off')

#             plt.show(block=False)
#             plt.draw()

#             # this pause is needed to ensure the drawing for some backends
#             plt.pause(0.001)

# Tracker Engine

In [ ]:
def process_event(z_queue, gray_queue, points3D_queue, ConfidenceIndex_queue, Confidence_queue, painter=None, seconds=150):
    global data_lock
    First = True
    timeout = .5
    wait_data_flag = False
    capture_frame = 45
    frame = 0
    t_end = time.time() + seconds # create a loop that will rsecondsthe given amount of time
    start_time = time.time()
    receive_time = 1
    num_of_frames = 0
    while time.time() < t_end:
        try:
            # try to retrieve an item from the queue
            # this will block until an item can be retrieved
            # or the timeout of 1 second is hit
            data_lock.acquire()
            zImage = z_queue.get(wait_data_flag, timeout) if z_queue else None
            grayImage_int32 = gray_queue.get(wait_data_flag, timeout) if gray_queue else None
            points3D = points3D_queue.get(wait_data_flag, timeout) if points3D_queue else None
            if First == True:
                ConfidenceIndex = ConfidenceIndex_queue.get(wait_data_flag, timeout) if ConfidenceIndex_queue else None
#             Confidence = Confidence_queue.get(wait_data_flag, timeout) if Confidence_queue else None
        except queue.Empty:
            # no new data, we wait 10 ms for receive new data
            data_lock.release()
            time.sleep(0.023)
            continue
        else:
            data_lock.release()
            if First == True:
                #RANSAM to get surface plane
                surface_plane, depthImg, plane_mask = get_clear_hand_mask.RANSAM(points3D, ConfidenceIndex, ransac_iteration = 500, inliner_threshold = 0.01)
                First = False
                painter.First = False
                print("firstly RANSAM")
                fs_time = time.time()
            else:
                #
                cs_time = time.time()
                s_time = time.time()
                depthImg = get_clear_hand_mask.get_depth_map(points3D, surface_plane)
#                 print('depth: %.4f'%(time.time() - s_time))

                #Canny edge map(infrared image) + threshold based edge map(depth image)
                t_time = time.time()
                Cannyedges, Threshold_based_edge, Edge_map, grayImage = get_clear_hand_mask.get_edge_map(grayImage_int32, depthImg)
                print('get clear hand mask time:', (time.time()-t_time))
                
                #Get hight region by Hight and record its position
                t_time = time.time()
                High_region_Image, high_region_list = get_clear_hand_mask.get_high_region(depthImg)
                print('get hight region time:', (time.time()-t_time))
                
                #Get Hand mask by Flood fill from high region position with Edge map and check wheather Flood_fill_with_edge_reasonable  
                t_time = time.time()
                Hand_mask_Image = get_clear_hand_mask.get_reasonable_Hand_mask(Edge_map, high_region_list, High_region_Image, points3D)
                print('get reasonable hand mask time:', (time.time()-t_time))
                
                # find fingertip
                t_time = time.time()
                cnt, contours_image, hand_center, fingertips = find_fingertips.find_fingertip(Hand_mask_Image, debug = True)
                print('find fingertip time:', (time.time()-t_time))
                
                # tracking user hand and fingertips
                t_time = time.time()
                hands_hand, fingertips_pos, tracking_iamge = tracking.hand_tracking(hand_center, fingertips, points3D, contours_image)
                send_fingertips_pos(fingertips_pos, points3D)
                print('hand tracking time:', (time.time()-t_time))
                
                # detecte fingers whenther touching the surface
                t_time = time.time()
                touched_flag, touching_detection_image = touched_detection.touching_detection(fingertips_pos, Hand_mask_Image, depthImg, contours_image)
                send_touch_events(touched_flag)
                print('touching detection time:', (time.time()-t_time))
                print('process time: %.4f s'%(time.time() - cs_time), 'fps: %.4f s'%(1/(time.time() - fs_time)))
                print()
                fs_time = time.time()

                frame = frame + 1
                num_of_frames = num_of_frames + 1
                if (frame > capture_frame):
                    # estimated time
                    receive_time = (time.time() - start_time)
                    frame = 0
                    start_time = time.time()
                if(receive_time != 0):
                    size = zImage.shape
                    input_fps = capture_frame
                    cv2.putText(img=touching_detection_image, text=('fps=%.1f'%(input_fps/receive_time)), org=(0, size[1]-75), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.25, color=(255,255,255))
                
                #Show original image and result image
#                 painter.paint(zImage,'zImage', False, True)
#                 painter.paint(grayImage*15,'Gray Image',True, True)
#                 painter.paint(depthImg,'Depth', False, True)
#                 painter.paint(Cannyedges,'Canny Edges',True)
#                 painter.paint(Threshold_based_edge,'Threshold_based Edge',True)
#                 painter.paint(Edge_map,'Edge map',True)
#                 painter.paint(High_region_Image,'High region Image',True)
#                 painter.paint(Hand_mask_Image,'Hand mask Image')
#                 painter.paint(contours_image,'Contours Image',False, True)
#                 painter.paint(tracking_iamge,'tracking Image',False, True)
                painter.paint(touching_detection_image,'touched Image', False, True)
#                 painter.paint(touching_detection_image,'touched Image', False)

                # Press Q on keyboard to exit 
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            
    print(num_of_frames)     

# Transform Data

In [ ]:
def send_fingertips_pos(fingertips, threed_points):
    data = ""
    temp = ['1 ']
    for i,tip in enumerate(fingertips):
        (u, v) = (int(tip[0]), int(tip[1]))
        temp.append("%d %d "%(u, v)) # speedup the string append
    data = data.join(temp)
    print('send fingertips pos:', data)
    socket_sender.send(data)
    
def send_touch_events(touch_flag):
    data = ""
    temp = ['2 ']
    for i,touch in enumerate(touch_flag):
        temp.append("%d "%(touch)) # speedup the string append
    data = data.join(temp)
    print('send touch flag:', data)
    socket_sender.send(data)

# Main

In [ ]:
def main ():
    global data_lock
    parser = argparse.ArgumentParser(usage=__doc__)
    add_camera_opener_options(parser)
    parser.add_argument("--seconds", type=int, default=15, help="duration to capture data")
    
    PlayBack = True
    if PlayBack:
        options = parser.parse_args(args=['--rrf','45fps.rrf','--seconds', '5'])
    else:
        options = parser.parse_args(args=['--seconds', '5'])
    opener = CameraOpener(options)
    cam = opener.open_camera()
    
    if PlayBack == False:
        cam.setUseCase('MODE_5_45FPS_500')#MODE_5_45FPS_500
    print("FrameRate: " ,cam.getFrameRate() ,"CurrentUseCase: " , cam.getCurrentUseCase())

    print_camera_info (cam)
    print("isConnected", cam.isConnected())
    print("getFrameRate", cam.getFrameRate())
    print(cam.getFilterLevel())

    z_queue = queue.Queue()
    gray_queue = queue.Queue()
    points3D_queue = queue.Queue()
    ConfidenceIndex_queue = queue.Queue()
    Confidence_queue = queue.Queue()
    l = MyListener(z_queue,gray_queue,points3D_queue,ConfidenceIndex_queue,Confidence_queue)
    l.set_lock(data_lock)
    cam.registerDataListener(l)
    cam.startCapture()

    process_threading = threading.Thread(target=process_event, args=(z_queue, gray_queue, points3D_queue, ConfidenceIndex_queue, Confidence_queue, l, 60))
    process_threading.start()
    
    print('timeout')
    process_threading.join() # wait the process threading finsish
    cam.stopCapture()
    z_queue.queue.clear()
    gray_queue.queue.clear()
    points3D_queue.queue.clear()
    ConfidenceIndex_queue.queue.clear()
    Confidence_queue.queue.clear()
    socket_sender.close_socket()
    
    wait_threads = l.get_store_threads()
    print('get store threads', wait_threads)
    for t in wait_threads:
        if(t.is_alive()):
            t.join()
    cv2.destroyAllWindows()
    print('finish')

data_lock = threading.Lock()
print('init data lock:', data_lock)
if (__name__ == "__main__"):
    main()

In [ ]:
#show plane edge
test = np.zeros((171,224), np.uint8)
l = 15
test[l:171-l,l:224-l] = 1
plt.imshow(test),plt.axis('off')
plt.show()

In [ ]:
#show plane edge
test = np.zeros((171,224), np.uint8)
h = 171
w = 224
l = 15
test[:l,:] = 1
test[h-l:h,:] = 1
test[:,:l] = 1
test[:,w-l:w] = 1
plt.imshow(test),plt.axis('off')
plt.show()